_TODO_ Don't forget to add Graphulo jars to your server-side Accumulo files! More info here: _TODO_ 

In [1]:
print("is restarted? this ought to be cell 1")

is restarted? this ought to be cell 1

In [2]:
using Revise
using D4M

┌ Info: Precompiling D4M [ca196bdc-a701-11e8-3d50-3b5cc8577617]
└ @ Base loading.jl:1192


Database capabilities loaded!


In [3]:
testD4Mver()

v test 6


In [4]:
dbinit()

In [6]:
DB = dbsetup("uno", "AccumuloConfig.jl")

26 Jun 2019 09:29:56,648 WARN - ClientConfiguration.loadFromSearchPath(227) -  Found no client.conf in default paths. Using default client configuration values.


D4M.DBserver("uno", "localhost:2181", "root", "secret", "BigTableLike", JavaCall.JavaObject{Symbol("edu.mit.ll.graphulo.MatlabGraphulo")}(Ptr{Nothing} @0x00000000051639e8))

In [67]:
A = DB["mytable_A"]
AT = DB["mytable_AT"]
B = DB["mytable_B"]

ls(DB)

Creating mytable_AT in uno


8-element Array{SubString{String},1}:
 "accumulo.metadata"   
 "accumulo.replication"
 "accumulo.root"       
 "mytable_A"           
 "mytable_AT"          
 "mytable_B"           
 "mytable_P"           
 "trace"               

In [81]:
row = [1,1,2,2]
column = [1,2,1,2]
values = [1,2,3,4]

AssocA = Assoc(row,column,values)
AssocAT = transpose(AssocA)
put(AT, AssocAT)
printFull(AT[:,:])

row = [1,1,2,2]
column = [1,2,1,2]
values = [5,6,7,8]

AssocB = Assoc(row,column,values)
put(B, AssocB)
printFull(B[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "1"  "3"
 "2"  "2"  "4"

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"  "2"
 "1"  "5"  "6"
 "2"  "7"  "8"

# Table Multiplication

In [64]:
delete(P)
P = tablemult(B, A, "mytable_P")
printFull(P[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "26"  "38"
 "2"  "30"  "44"

26 Jun 2019 12:47:40,853 DEBUG - Graphulo.TwoTable(770) -   :%00; [1] -> 8 entries processed


In [65]:
printFull(P[:,:])

3×3 Array{Union{AbstractString, Number},2}:
 ""   "1"   "2" 
 "1"  "26"  "38"
 "2"  "30"  "44"

In [63]:
Am=[1 3
2 4]
Bm = [5 6
7 8]
print(Am*Bm)
print(2*Am*Bm)
print(3*Am*Bm)
print(3*Am*Bm + transpose(Bm)*transpose(Am))

[26 30; 38 44][52 60; 76 88][78 90; 114 132][104 128; 144 176]

We can add filters too --

In [ ]:
C = tablemult(A, B, "mytable_C")